In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pathresults = "../results/"
#pathresults = "/DUMUXDune27/DUMUX/dumux-rosi/python/paperSc/results/"
data_file_delimiter = ","
from plotnine import *
import re

In [8]:
def getVal(mypath, filename, header="infer",names = None):
    fullpath = pathresults +mypath+filename
    if names is None:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header).dropna(how= "all", axis=1)
    else:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header, names = names).dropna(how= "all", axis=1)
    return cst
def get_last_line_as_numpy_array(file_path, dtype = float):
    with open(pathresults +file_path, 'r') as file:
        # Read all lines
        lines = file.readlines()

        # Check if there are any lines in the file
        if not lines:
            raise ValueError("The file is empty")

        # Get the last line
        last_line = lines[-1].strip()

        # Split the last line by commas and convert to NumPy array
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)

        return array_from_last_line

In [9]:

def list_files_with_prefix(folder_path, prefix):
    file_names = []
    for file_name in os.listdir(pathresults+folder_path):
        if file_name.startswith(prefix):
            file_names.append(file_name)
    return file_names
def getCylIdx(folder_path):
    namelist = list_files_with_prefix(folder_path, "Cyl_watercontent_")
    idxlist = np.full(len(namelist),-1,dtype = int)
    for nn, nl in enumerate(namelist):
        match = re.search(r'\d+', nl)
        number_str = match.group()
        number_int = int(number_str)
        idxlist[nn] = number_int
    return idxlist

In [14]:
# cylinder max konz (per cell)
def getData_(name, cid,gId,path2file):
    fileName = "Cyl_cellVol_"+str(gId)+".txt"
    cVol = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
    if cid <= 9:
        if cid <= 2 and cid > 0:
            fileName = "Cyl_watercontent_"+str(gId)+".txt"
            theta = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
            cVol *= theta
        fileName = name+str(gId)+".txt"
        Q1 = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
        konz = sum(Q1)/sum(cVol) #mol/cm3 or cm3/cm3
        return konz
    return sum(cVol)

def getData():
    
    nToGet = 9+3+1 #elements + 3d coordinates of y node+vol
    nKonz = 9
    dictXYZ = {9:'X',10:'Y',11:'Z'}
    GiniAll = [[[] for i in range(2)] for ii in range(nToGet)]
    for pp, pSet in enumerate([10,29]):
        print(pSet,end =", ")

        path2file = 'none_55_'+str(pSet)+'_10to11_20mn_0s_5/'
        #nodes_X =get_last_line_as_numpy_array(path2file+"nodes_X.txt", dtype =float)
        #nodes_Y =get_last_line_as_numpy_array(path2file+"nodes_Y.txt", dtype =float)
        #nodes_Z =get_last_line_as_numpy_array(path2file+"nodes_Z.txt", dtype =float)
        #dictXYZ = {10:nodes_X,11:nodes_Y,12:nodes_Z}
        rr = getCylIdx(path2file)

        for cid in range(nKonz):
            Ginits = np.full(len(rr),np.nan)
            for idx, gId in enumerate(rr):
                if cid == 0:
                    namedata = "Cyl_watercontent_"
                elif cid <= nKonz:
                    namedata = "Cyl_content"+str(cid)+"_"
                Ginits[idx] = getData_(namedata, cid, gId,path2file)                
            GiniAll[cid][pp] = Ginits
        for cid in range(nKonz,nToGet-1):
            nodes_ =get_last_line_as_numpy_array(path2file+"nodes_"+dictXYZ[cid]+".txt", dtype =float)
            GiniAll[cid][pp] = nodes_[rr+1]#seg idx to y-idx
        cid = nToGet -1
        Ginits = np.full(len(rr),np.nan)
        for idx, gId in enumerate(rr):
            Ginits[idx] = getData_('', cid, gId,path2file)
        GiniAll[cid][pp] = Ginits
    return GiniAll #cid pSet rr

In [15]:
GiniAll = getData()

10, 29, 

In [17]:
print(len(GiniAll),len(GiniAll[0]),len(GiniAll[0][0]),len(GiniAll[0][1]))

13 2 232 236


In [13]:
print(np.array(GiniAll[0]).shape)

(99,)


/tmp/ipykernel_132/3067407154.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [19]:
GiniAll[12][0]

0.26315789473684204